<a href="https://colab.research.google.com/github/pippi-chen/Info-Challenges/blob/main/SBA_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
# the packages that have already been used
import pandas as pd
import numpy as np
import seaborn as sn

# the packages that haven't been used yet
import matplotlib 
import matplotlib.pyplot as plt
import sklearn
import io
from sklearn import preprocessing

# All loans to Georgia businesses that remain in the PPP database (ppp_applicants_ga_full.csv)

In [89]:
# import data
filepath = "ppp_applicants_ga_full.csv"
data = pd.read_csv(filepath)
df = pd.DataFrame(data)

## Data Prepocessing

In [90]:
# detect NA
na = df.isna().any()

# find the columns with NA 
NA_columns = df.columns[na].tolist()
NA_columns

['name',
 'address',
 'naics_code',
 'business_type',
 'congressional_district',
 'undisbursed_amount',
 'project_county_name',
 'loan_status_date',
 'forgiveness_amount',
 'forgiveness_date']

In [91]:
# summary of the data (numerical variables)
df.describe(include=[np.number])

,amount,naics_code,jobs_retained,loan_number,sba_office_code,term,sba_guaranty_percentage,initial_approval_amount,current_approval_amount,undisbursed_amount,servicing_lender_location_id,originating_lender_location_id,forgiveness_amount
count,5.538280e+05,551488.000000,553828.000000,5.538280e+05,553828.000000,553828.000000,553828.0,5.538280e+05,5.538280e+05,553713.000000,553828.000000,553828.000000,3.711970e+05
mean,4.493328e+04,566579.014484,4.929097,5.560550e+09,405.003886,51.114297,100.0,4.531962e+04,4.493328e+04,11.868519,302901.102525,302883.585810,5.149423e+04
std,1.859896e+05,180415.096400,18.447953,2.571168e+09,1.421909,15.527003,0.0,1.899497e+05,1.859896e+05,603.065725,216045.590952,216334.440538,2.077751e+05
min,1.000000e+00,111110.000000,0.000000,1.000019e+09,101.000000,5.000000,100.0,0.000000e+00,1.000000e+00,-103592.000000,27.000000,27.000000,1.000000e-02
25%,1.035400e+04,484110.000000,1.000000,3.377540e+09,405.000000,60.000000,100.0,1.040000e+04,1.035400e+04,0.000000,85712.000000,81965.000000,1.033288e+04
50%,2.020800e+04,541613.000000,1.000000,5.592663e+09,405.000000,60.000000,100.0,2.023100e+04,2.020800e+04,0.000000,382926.000000,317954.000000,2.039047e+04
75%,2.083300e+04,722320.000000,2.000000,7.787251e+09,405.000000,60.000000,100.0,2.083300e+04,2.083300e+04,0.000000,524612.000000,524612.000000,2.344273e+04
max,1.000000e+07,999990.000000,500.000000,9.999007e+09,912.000000,60.000000,100.0,1.000000e+07,1.000000e+07,129375.000000,533179.000000,533179.000000,1.013667e+07


In [92]:
# summary of the data (categorical variables)
df.describe(include=[object]) 

,name,state,address,city,zip,business_type,date_approved,lender,congressional_district,processing_method,loan_status,servicing_lender_name,servicing_lender_address,servicing_lender_city,servicing_lender_state,servicing_lender_zip,rural_urban_indicator,hubzone_indicator,business_age_description,project_city,project_county_name,project_state,project_zip,originating_lender_city,originating_lender_state,loan_status_date,lmi_indicator,forgiveness_date
count,553827,553828,553827,553828,553828,553787,553828,553828,553823,553828,553828,553828,553828,553828,553828,553828,553828,553828,553828,553828,553822,553828,553828,553828,553828,280589,553828,371197
unique,444162,1,429791,2762,229592,23,237,1194,38,2,3,1184,1232,857,53,1239,2,2,5,2777,171,10,229694,861,53,355,2,270
top,MICHAEL WILLIAMS,GA,1445 Woodmont Ln NW,Atlanta,30024,Sole Proprietorship,2020-05-01,"Prestamos CDFI, LLC",GA-05,PPP,Paid in Full,"Harvest Small Business Finance, LLC",24422 Avenida de la Carlota Suite 232,Laguna Hills,GA,92653,U,N,Existing or more than 2 years old,Atlanta,FULTON,GA,30024-0001,Phoenix,GA,2021-09-25,N,2021-09-07
freq,51,553828,238,67488,1031,220996,22400,52038,75046,434959,280245,60048,60048,60048,111291,60048,429786,374604,511266,67571,96693,553814,915,52041,118554,12591,365697,8575


In [93]:
# correct the data type
# numerical to categorical
df.naics_code = df.naics_code.apply(str)
df.loan_number = df.loan_number.apply(str)
df.sba_office_code = df.sba_office_code.apply(str)
df.servicing_lender_location_id = df.servicing_lender_location_id.apply(str)
df.originating_lender_location_id = df.originating_lender_location_id.apply(str)

In [94]:
# address the NA values

# undisbursed_amount: change NA into 0
df.undisbursed_amount = df.undisbursed_amount.fillna(0)

# project_county_name: add the correct county name
df[df.project_county_name.isna()] # find the observations records of the NA value
df.project_county_name.fillna("LEE", limit = 1, inplace = True) # observation 39150: project_county_name = LEE
df.project_county_name.fillna("FULTON", limit = 1, inplace = True) # observation 88268: project_county_name = FULTON
df.project_county_name.fillna("COBB", limit = 1, inplace = True) # observation 177572: project_county_name = COBB
df.project_county_name.fillna("COLQUITT", limit = 1, inplace = True) # observation 239504: project_county_name = COLQUITT
df.project_county_name.fillna("GWINNETT", limit = 1, inplace = True) # observation 279180: project_county_name = GWINNETT
df.project_county_name.fillna("COBB", limit = 1, inplace = True) # observation 396988: project_county_name = COBB

# drop name
df = df.drop(columns="name" ,axis=1)

# drop forgiveness_amount
df = df.drop(columns="forgiveness_amount" ,axis=1)

# drop forgiveness_data
df = df.drop(columns="forgiveness_date" ,axis=1)

# Loans to Georgia businesses that were removed from the PPP database (ppp-removed-ga.xlsx)

In [95]:
filepath1 = "ppp-removed-ga.xlsx"
data1 = pd.read_excel(filepath1)
df1 = pd.DataFrame(data1)

## Data Prepocessing

In [96]:
# detect NA
na1 = df1.isna().any()

# find the columns with NA 
NA_columns1 = df1.columns[na1].tolist()
NA_columns1

['undisbursed_amount',
 'loan_status_date',
 'forgiveness_amount',
 'forgiveness_date']

In [97]:
# summary of loan_status_date (because I cannot find this variable on the below summaries)
df1.loan_status_date.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


count                   23331
unique                    163
top       2021-05-22 00:00:00
freq                     2173
first     2020-04-06 00:00:00
last      2021-06-29 00:00:00
Name: loan_status_date, dtype: object

In [98]:
# summary of the data (numerical variables)
df1.describe(include=[np.number])

,amount,naics_code,jobs_retained,loan_number,sba_office_code,term,sba_guaranty_percentage,initial_approval_amount,current_approval_amount,undisbursed_amount,servicing_lender_location_id,originating_lender_location_id,forgiveness_amount,forgiveness_date
count,2.583600e+04,25836.000000,25836.000000,2.583600e+04,25836.0,25836.000000,25836.0,2.583600e+04,2.583600e+04,2.583500e+04,25836.000000,25836.000000,0.0,0.0
mean,1.999880e+04,585683.342545,1.280423,6.143559e+09,405.0,59.892708,100.0,2.003320e+04,1.999880e+04,1.826408e+04,376921.468068,377062.431801,NaN,NaN
std,6.836528e+04,179477.217495,3.954098,2.640659e+09,0.0,1.962436,0.0,6.837832e+04,6.836528e+04,6.845939e+04,173107.376794,173039.576934,NaN,NaN
min,1.020000e+02,111110.000000,1.000000,1.000249e+09,405.0,24.000000,100.0,1.020000e+02,1.020000e+02,0.000000e+00,2408.000000,2408.000000,NaN,NaN
25%,1.479100e+04,484110.000000,1.000000,3.972551e+09,405.0,60.000000,100.0,1.494075e+04,1.479100e+04,1.000000e+04,188361.000000,188361.000000,NaN,NaN
50%,2.051000e+04,561499.000000,1.000000,6.577589e+09,405.0,60.000000,100.0,2.051500e+04,2.051000e+04,2.028200e+04,509316.000000,509316.000000,NaN,NaN
75%,2.083200e+04,811211.000000,1.000000,8.379621e+09,405.0,60.000000,100.0,2.083250e+04,2.083200e+04,2.083200e+04,529471.000000,529471.000000,NaN,NaN
max,1.000000e+07,999990.000000,495.000000,9.998679e+09,405.0,60.000000,100.0,1.000000e+07,1.000000e+07,1.000000e+07,531105.000000,533279.000000,NaN,NaN


In [99]:
# summary of the data (categorical variables)
df1.describe(include=[object]) 

,name,state,address,city,zip,business_type,lender,congressional_district,processing_method,loan_status,servicing_lender_name,servicing_lender_address,servicing_lender_city,servicing_lender_state,servicing_lender_zip,rural_urban_indicator,hubzone_indicator,business_age_description,project_city,project_county_name,project_state,project_zip,originating_lender_city,originating_lender_state,lmi_indicator
count,25836,25836,25836,25836,25836,25836,25836,25836,25836,25836,25836,25836,25836,25836,25836.0,25836,25836,25836,25836,25836,25836,25836,25836,25836,25836
unique,25272,1,24743,670,22287,17,89,14,2,2,89,89,80,24,88.0,2,2,3,670,159,1,22337,80,24,2
top,ANTHONY WILLIAMS,GA,1445 Woodmont Ln NW,Atlanta,30318-2866,Sole Proprietorship,"Capital Plus Financial, LLC",GA-13,PPP,Active Un-Disbursed,"Capital Plus Financial, LLC",2247 Central Drive,Bedford,TX,76021.0,U,N,Existing or more than 2 years old,Atlanta,FULTON,GA,30318-2866,Bedford,TX,N
freq,7,25836,7,4093,81,17093,6269,4044,21140,23331,6269,6269,6269,6712,6269.0,20480,19511,25799,4093,4561,25836,81,6269,6713,15463


In [100]:
# correct the data type
# numerical to categorical
df1.naics_code = df1.naics_code.apply(str)
df1.loan_number = df1.loan_number.apply(str)
df1.sba_office_code = df1.sba_office_code.apply(str)
df1.servicing_lender_location_id = df1.servicing_lender_location_id.apply(str)
df1.originating_lender_location_id = df1.originating_lender_location_id.apply(str)
df1.forgiveness_date = df1.forgiveness_date.apply(str)

In [101]:
# address the NA values

# undisbursed_amount: change NA into 0
df1.undisbursed_amount = df1.undisbursed_amount.fillna(0)

# drop name
df1 = df1.drop(columns="name" ,axis=1)

# drop forgiveness_amount
df1 = df1.drop(columns="forgiveness_amount" ,axis=1)

# drop forgiveness_data
df1 = df1.drop(columns="forgiveness_date" ,axis=1)

# Create the variable "Removed"

In [102]:
df["removed"] = 0
df1["removed"] = 1

In [103]:
# Save datasets to new excel files

df.to_csv('new_ppp_applicants_ga_full.csv')
df1.to_excel('new_ppp-removed-ga.xlsx')